# Error Correcting Codes Encoding Study

The goal of this study is to understand options to the popular one-hot encoding. There are many sides to each story (no, not only two), on those sides are: 

- I never liked one-hot encoding (and is been more than a decade since I first used it, so the disgust might never go out);
- I don't like how neural networks are treated and should always be end to end learning (no they should not, they should be more complex architectures, many already in research literature)
- There are priors 
- Each type of input should  (and HAS in nature) its own priors, which are adapted to *facilitate* the learning, no we should not do everyhting inside a NN, we should give as input something that has priors that facilitate learning (and might or might not later save processing power during operations)


On the priors, many have already shown good results, the most remarcable prior are: Convolutional Neural Networks, MAC Networks, LSTMs, others are more subtle, like (remember citation here ...) adding a coordinate system to the input image as an (or many) extra channel(s). There are many more that I think are worth exploring and adding to the literature, even if they don't give good results. 
On those priors there are many that we not only know, but also we have specialized hardware that is perfectly adapted
* time and space -> this we can encode and add it as extra channels
* Different transforms (Fourier, Laplace, Wavelets, ...)
* spikes (borders in images)
* ....

There is another idea behind the ECCs, is that we usually can 'feel' that something is not right or missing, what about giving NNs a way of having an extra element that would allow for this 'feeling'? 

The idea behind this is that I don't agree with one-hot encoding the way is used today, not because it does not work, but because it imposes a few limits that I don't want to deal with at first

* We know the actual number of values to encode (with words this is not necessary true)
* We have a sample data to train the encoding

This limits us in several ways; for example, for training on a domain, the encoder will depend on that domain only. If there are under-represented values (such as words that don't appear or are new later, or changing domain) this limits the encoding possibliities. A better idea will be to be able to encode everything even if the internal representations have not yet learned to use those simbols.

I want to be able to do a separation ebtween the *possibility*  of representing a value, and the learning of that concept.

The first and biggest limitation of one-hot encoding is that does not allow to represent values that are not accepted.

As some other parts of this study have already focused on integer value representations, arbitrary function representation (although with limitted success on the fourier inspired encodings) this study is more focused on being able to represent correctly all the values of utf-8, basically doing a first binary representation that will be given as input to an OVERFITTED encoder. 

The reasoning behind this is:


* The origin domain is all text
* UTF-8 can represent all text in all languages including some extra elements
* let's use UTF-8 as the origin domain
* Create an encoder that can deal with ANY and ALL input in the origin domain
* the encoded values can later be used

As text should be always correctly reconstructed in the presence of noise, I want to imagine now a Neural Network like a REALLY NOISY channel. For this using (Forward) ECCs is one way of thinking in this medium
Then the tests that I intend to do is:

* Create an autoencoder that OVERFITS to all the data


One idea that I have been dealing with my head for the past 3-4 years is that we are thinking overfitting the wrong way, and we can actually use it well, but we have to learn how.

I think that here is the first time I actually find a way of doing it in a useful way

The idea is to overfit so it generates an smaller encoding vector than the one in the input. Autoencoders are good to do this test.

The other idea is that if the autoencoder can NOT do this, then the encoding ideas that I will try are BAAAD and I should feel BAAAD. In this case ... just go to the drawing table and think of other things.

On the other side, if this works, this means that FINALLY I can go on the next stage, that is building the predictors first basic ones (LSTMs, HMMs, Time Convolutions), then with meta-learning and later with my still too fresh idea on neural databases. 

One interesting thing I want to find out about Error Correcting Codes (ECCs) is if they are actually useful in the output decoding, as they should be adding *explicit* redundancy to the input and also to the output.

The other thing about ECCs is that we can pile them up, for example, one (or many codes) to representa a symbol (for example the value *'€'* ) and then use convolutional or turbo codes for the *temporal* encoding/decoding part, this means that we not only add priors to the intantaneous input, but also to the temporal dimension, which is something I really want to explore (and this should facilitate fixing and correcting "channel errors")

I don't deal here with *erasure* error types, but that is a possibility later.


In [ ]:
import numpy as np
import commpy
import bitarray as ba
import struct
import sys
import pickle
# import binascii
from bitstring import BitArray, BitStream


In [ ]:
sys.byteorder

In [ ]:
c = '€'.encode()

In [ ]:
c

In [ ]:
a = 'a'

In [ ]:
'a'.encode()[0]

In [ ]:
len(bytearray('a'.encode()))

In [ ]:
zero = BitArray(b'\x00\x00\x00\x00')
b = BitArray(c)

In [ ]:
b

In [ ]:
b.tobytes()

In [ ]:
int.from_bytes(c, byteorder='big')

In [ ]:
32 - b.len

In [ ]:
int.from_bytes(c, byteorder='big') >> 1

In [ ]:
for i in range ((32 - b.len)//8):
    b.prepend(b'\x00')

In [ ]:
b.len

In [ ]:
b

In [ ]:
32 - b.len

In [ ]:
a = 256
a.bit_length()

In [ ]:
'€'.encode()[1]

In [ ]:
# Bit of a whacky hack and for sure not the most efficient one, but it just works for what I want

def prepend_zeros(s, n):
    return '0'* (n - len(s))+s

def get_strbintable(n):
    bl = n.bit_length() - 1  # because n is actually never used
    lines = [ ' '.join(i for i in prepend_zeros("{0:b}".format(l), bl)) for l in range(n)]
    return lines

def get_npbintable(n):
    bins = np.fromstring('\n'.join(get_strbintable(n)), dtype='int32', sep=' ')
    bins = bins.reshape([n, -1])
    return bins
    

Since the entire utf-8 univers is NOT the entire $2^{32}$ domain, but there are limitations explained in [the utf-8 description](https://en.wikipedia.org/wiki/UTF-8)

| Number of bytes | Bits for code point | First code point | Last code point | Byte 1   | Byte 2   | Byte 3   | Byte 4   |
|----------------|--------------------|-----------------|----------------|----------|----------|----------|----------|
| 1              | 7                  | U+0000          | U+007F         | 0xxxxxxx |          |          |          |
| 2              | 11                 | U+0080          | U+07FF         | 110xxxxx | 10xxxxxx |          |          |
| 3              | 16                 | U+0800          | U+FFFF         | 1110xxxx | 10xxxxxx | 10xxxxxx |          |
| 4              | 21                 | U+10000         | U+10FFFF       | 11110xxx | 10xxxxxx | 10xxxxxx | 10xxxxxx |

I'll then compute different table parts and do an append when needed

The thing is that the number of elements in the table should be at most $2^{21}$, I need to create a sort of index that can handle the 4 cases.
It seems I'll have to create 4 different conversion tables.




In [ ]:
# this part makes sure to encode as bin
eye4 = np.eye(4)
eye64 = np.eye(64)
eye256 = np.eye(256)

# code for 7 bits, Byte 1 of utf-8
code_b7 = np.append(np.zeros([2**7, 1]), get_npbintable(2**7), axis=1)

# code for 6 bits, Byte 2 to 4 of utf-8 -> this is going to be used later for all the other values
code_b6 = np.append(np.append(np.ones([2**6, 1]), np.zeros([2**6, 1]), axis=1), 
                    get_npbintable(2**6), axis=1)

# code for 5 bits, Byte 1 of 
code_b5 = np.append(np.append(np.ones([2**5, 2]), np.zeros([2**5, 1]), axis=1), 
                    get_npbintable(2**5), axis=1)

# code for 4 bits, Byte 2 to 4 of utf-8 -> this is going to be used later for all the other values
code_b4 = np.append(np.append(np.ones([2**4, 3]), np.zeros([2**4, 1]), axis=1), 
                    get_npbintable(2**4), axis=1)

# code for 3 bits, Byte 2 to 4 of utf-8 -> this is going to be used later for all the other values
code_b3 = np.append(np.append(np.ones([2**3, 4]), np.zeros([2**3, 1]), axis=1),
                    get_npbintable(2**3), axis=1)

def encode_utf8(l):
    el = l.encode()
    code = np.zeros(36)  # 32 is the size of the input code + 4 of the extra redundancy
    nbytes = len(el)
    # assert( 0<nbytes && nbytes<=4)
    assert(nbytes<=4)
    bin4 = eye4[nbytes-1]  # this adds redundant knowledge about the  part
    # this is ugly but explicit, for the moment is good enough and I can see what is
    code[:4] = bin4
    if nbytes == 1:
        code[4:12] = code_b7[el[0]& 0b01111111 ]
    elif nbytes == 2:
        code[4:12] = code_b5[el[0] & 0b00011111 ]
        code[12:20] = code_b6[el[1] & 0b00111111]
    elif nbytes == 3:
        code[4:12] = code_b4[el[0] & 0b00001111]
        code[12:20] = code_b6[el[1] & 0b00111111]
        code[20:28] = code_b6[el[2] & 0b00111111]
    elif nbytes == 4:
        code[4:12] = code_b3[el[0] & 0b00000111]
        code[12:20] = code_b6[el[1] & 0b00111111]
        code[20:28] = code_b6[el[2] & 0b00111111]
        code[28:36] = code_b6[el[3] & 0b00111111]
    else:
        raise Exception("Bad input, input has to have 1 to 4 input bytes")
    return code


# TODO I need to find a more efficient way of doing this that could make this as vector or matrix operations instead
def encode_utf8_multihot(c):
    e_c = list(c.encode())
#     code = np.zeros(36)  # 32 is the size of the input code + 4 of the extra redundancy
    nbytes = len(e_c)
    # assert( 0<nbytes && nbytes<=4)
    assert(nbytes<=4)
    bin4 = eye4[nbytes-1]  # this adds redundant knowledge about the  part
    # this is ugly but explicit, for the moment is good enough and I can see what is
#     code[:4] = bin4
    # max size of each part of the code
    # I will treat the first byte as always 8 bits, this will make it easier to decode later and adds aditional information
    # this has an extra benefit, when a code is there only certain regions will become 1 giving an extra hint to the network
    # maxsizes = [2**8, 2**6, 2**6, 2**6]
    code = np.zeros(4 + (2**8) + 3*(2**6))
    masks = [0xff, 0x3f, 0x3f, 0x3f]
    indices = [256+4, 64+256+4, 2*64 + 256+4, 3*64 + 256+4]
    maxsizes = [eye256, eye64, eye64, eye64]
    code[:4] = bin4
    prev_i = 4
    for i,n,e,m in zip(indices[:nbytes], e_c, maxsizes[:nbytes], masks[:nbytes]):
        code[prev_i:i] = e[n & m]  #masking
        prev_i = i
    return code

def encode_utf8_ecc(l):
    # TODO ...
    el = l.encode()
    code = np.zeros(36)  # 32 is the size of the input code + 4 of the extra redundancy
    nbytes = len(el)
    # assert( 0<nbytes && nbytes<=4)
    assert(nbytes<=4)
    bin4 = eye4[nbytes-1]  # this adds redundant knowledge about the  part
    # this is ugly but explicit, for the moment is good enough and I can see what is
    raise NotImplementedError("not implemented yet")


In [ ]:
list(zip([1,2,3,4], (1,2,3,4), (1,2,3,4)))

In [ ]:
el = '€'.encode()
'{0:b}  {1:b}  {2:b}'.format(el[0], el[1], el[2])

In [ ]:
encode_utf8('€')

In [ ]:
el = 'á'.encode()
'{0:b}  {0:b}'.format(el[0], el[1])

In [ ]:
encode_utf8('á')

In [ ]:
encode_utf8_multihot('€').shape

In [ ]:
l1 = [97,0,0,0]
l2 = [98,0,0,0]
l3 = [99,0,0,0]
l4 = [254,200,210,210]

In [ ]:
str(bytes([0x20]),'utf-8')

In [ ]:
d = {bytes([1,2,3,4]): 'lala '}

In [ ]:
d

In [ ]:
2**21 # this should be enough to make the entire utf-8 encoding ... and much more

In [ ]:
%%time

tt10 = get_npbintable(2**21)

In [ ]:
tt10[101]

In [ ]:
# Dense binary input codes

# code for 7 bits, Byte 1 of utf-8
code_b7 = get_npbintable(2**7)
t_zeros = np.zeros([2**7, 1])
code_b7 = np.append(t_zeros, code_b7, axis=1)

# code for 6 bits, Byte 2 to 4 of utf-8 -> this is going to be used later for all the other values
code_b6 = get_npbintable(2**6)
t_b6 = np.append(np.ones([2**6, 1]), np.zeros([2**6, 1]), axis=1)
code_b6 = np.append(t_b6, code_b6, axis=1)

# code for 5 bits, Byte 1 of 
code_b5 = get_npbintable(2**5)
t_b5 = np.append(np.ones([2**5, 2]), np.zeros([2**5, 1]), axis=1)
code_b5 = np.append(t_b5, code_b5, axis=1)

# code for 6 bits, Byte 2 to 4 of utf-8 -> this is going to be used later for all the other values
code_b4 = get_npbintable(2**4)
t_b4 = np.append(np.ones([2**4, 3]), np.zeros([2**4, 1]), axis=1)
code_b4 = np.append(t_b4, code_b4, axis=1)

# code for 6 bits, Byte 2 to 4 of utf-8 -> this is going to be used later for all the other values
code_b3 = get_npbintable(2**3)
t_b3 = np.append(np.ones([2**3, 4]), np.zeros([2**3, 1]), axis=1)
code_b3 = np.append(t_b3, code_b3, axis=1)

# 4 bits
b4 = get_npbintable(2**4)
eye4 = np.eye(4)

In [ ]:
eye4

In [ ]:
np.eye(16)

In fact ... it seems that I can just chunk the utf-8 value in chunks and do one-hot per different parts:
- there are only 4 segment ranges, that can be coded in one-hot also add there either hamming or other ECC
- the largest value is for 7 bits -> 128 values
- the others contain 6 bits -> 64 values
The prefix of each can be taken away and replaced by the initial one-hot

So a complete code would be
$ 4 + 128 + 64 + 64 + 64 = 324 $

plus the ECC parity bits

Instead of having dimension 1,112,064 to encode any utf-8 value.

The encoder is  much simpler than I thought for this case, later I can add ECC for each, knowing that there is only one active bit in each row, this makes the task easier.

This embedding can stil be reduced but should be sparse enough already to make a good input

In [14]:
4 + 128 + 64 + 64 + 64 

324

In [ ]:
4 + 128 + 64 + 64 + 64 

In [ ]:
c

In [ ]:
np.fromstring('0 0 1 0 1 0 0 1', dtype=bool, sep=' ')

In [ ]:
np.fromstring('00101001', dtype=bool)  # there seems to be an issue here on numpy ...

In [ ]:
np.fromstring('0 0 1 0 1 0 0 1', dtype=int, sep=' ')

In [ ]:
bins = np.fromstring('\n'.join(get_strbintable(16)), dtype='<H', sep=' ')

In [ ]:
bins.reshape([16,-1])

In [ ]:
np.array(get_strbintable(16))


I tried to do some things about the first part of the code, turning bytes to a numpy array, but seems that the most efficient way would be a table (numpy 2d array that has as index the int value of the input and there the array in that position is the binary code, this can already include the first pass to make a one hot of every N bits (maybe every 4, so there are not so many initial values ), this matrix could already have pre-computed the ECC ...

For the ECC, I stil don't decide if making it by chunks of input bits, or by all the values, I guess that by all should do, but maybe is easier to compute them reshaping the input arrays to the code in use (example for Golay [24,12,8] will do for every 12 input bits) 

The idea is to not completely get rid of one-hot encoding, but to limit it to parts of the input vector code restricting the size of the domain

In [ ]:
# number of parameters for a one-hot by chunks encoding:
chunk_sizes = [4, 5, 6, 8, 12]
n_params = []
for c in chunk_sizes:
    n_params.append((c, (32 // c) * 2**c))

In [ ]:
n_params

Maybe for my tests up to chunks of size 6 should be acceptable (I still need to add the next ECC)

The next code can be:
- Repetition (x3)
- Hamming
- Golay
- Reed Solomon
- Latin Square 
- AN Correcting

Here some thoughts about the codes:

Repetition: this has the disadvantage of giving a redundancy that is quite obvious besides the low power of reconstruction of catastrofic errors. Just repeating does not necessarilly adds to a neural network another perspective at the input. Might be worth trying, but for the moment I'm not interested in it.

Hamming: it can correct one error (Hamming 7,4), adding 3 out of 4 bits. With an extra bit it can detect up to 2 errors with an extra parity bit.

Golay: might serve well enough for my first tests as it ads not too much overhead (duplicates the number of elements) for an interesting error correction (up to 3 bits from each 12, so one fourth).


There is one difference in the focus from this analysis and telecomunications (or any other domain with noisy channel), here I'm interested not in the code Rate (ammount of information sent vs ammount of actual bits sent) but in giving as input to the NN some form of non necessary evident redundancy that it could use, and having more ways to correct the output if a single mistake is made during the output extrapolation, I want ot check this part.

Thinking a bit more about auto-encoders, it might not be the best idea to start there as it might not give any useful information ... I still have to try some things, might give it a try if it is quick enough to build it once I have the input code.


For efficiency what I will do is build from the beginning the encoding table, for the decoding, I still need to think it thorugh.


In [ ]:
import torch

emd = torch.nn.Embedding(2**10, 300)

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, emd.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])


In [ ]:
params

In [ ]:
# from https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/7
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
count_parameters(emd)

The embedding layer is a fully connected layer ... this means a LOT of parameters

To be able to do an effective one-hot of all utf-8 would be:

In [ ]:
for i in [50,100,200,300]:
    print(i, 1112064 * i)

Which means I don't want to train that layer ... it would not even fit in my GPU

There is another thing, the embedding layer learns from the sample input, this means that it will ignore all values that don't appear or are underrepresented (a know issue). My goal is to deal with this with meta-learning techniques, but always being able to keep adding new inputs.

So I want a few encoders to try:

- chunked one-hot + hamming error correction of the first element
- binary like encoding only (will be done per byte to avoid making a table that is too big)
- binary like encoding with ECCs
- binary like encoding but added one-hot by each 4 bits (total 16 * 8 -> 128 bits)
- binary like encoding but added one-hot by each (4|12) bits plus ECC (total (16 * 8) + overload), hamming=224, golay=256




In [ ]:
128  + 32*3

Well, for the moment I think that the thing that I need can be done with multihot encoding, this is easier to decode with multiple log softmax.

For the most complex case of ECC, there is much more work to do for the decoding and even though I do have an idea of the encoding part I don't know yet how to do the decoding in a NN yet.



In [ ]:
# decoding utf8 encoded

nums = [0x01, 0x02, 0x03, 0x04]
masks = [0xf0, 0xe0, 0xd0, 0xc0]

In [ ]:
np.array(nums) | np.array(masks)

In [ ]:
bytes([127])

In [1]:
from utf8_encoder import *
import pickle

In [2]:
tables = create_tables(segments=3)

number of codes =  59328
number of code_exceptions =  4224


In [2]:
tables = create_tables()  # 4 segments by default

number of codes =  1107904
number of code_exceptions =  790656


In [4]:
len(tables)

5

In [3]:

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [4]:
np.save("utf8_code_matrix_3seg", tables[0])

In [5]:
save_obj(tables[1], "txt2code_3seg")
save_obj(tables[2], "code2txt_3seg")
save_obj(tables[3], "txt2num_3seg")
save_obj(tables[4], "num2txt_3seg")

In [6]:
t2c = tables[1]
c2t = tables[2]
n2t = tables[4]
t2n = tables[3]

In [9]:
len(t2n.keys()), len(n2t.keys()), len(tables[1].keys()), len(tables[2].keys()),

(59328, 59328, 59328, 59328)

Although Wikipedia says:

  UTF-8 is a variable width character encoding capable of encoding all 1,112,064[1] valid code points in Unicode using one to four 8-bit bytes.
  
We have managed to encode only 1107904 codes, so we are missing somehow 4160 codes that python can't encode from bytes:

In [13]:
1112064 - 1107904

4160

In [1]:
128 + (2**5 * 2**6)+ (2**4 * (2**6)**2) + (2**3 * (2**6)**3)

2164864

In [2]:
2**21 + 2**16 + 2**11 + 2**7

2164864

In [3]:
print("indices: ", 128, (128 + 2**5 * 2**6), (128 + 2**4 * (2**6)**2), (128 + 2**3 * (2**6)**3) )

indices:  128 2176 65664 2097280


In [7]:
# from:  https://stackoverflow.com/questions/7971618/python-return-first-n-keyvalue-pairs-from-dict
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [27]:
take(100, n2t.items())

[(0, '\x00'),
 (1, '\x01'),
 (2, '\x02'),
 (3, '\x03'),
 (4, '\x04'),
 (5, '\x05'),
 (6, '\x06'),
 (7, '\x07'),
 (8, '\x08'),
 (9, '\t'),
 (10, '\n'),
 (11, '\x0b'),
 (12, '\x0c'),
 (13, '\r'),
 (14, '\x0e'),
 (15, '\x0f'),
 (16, '\x10'),
 (17, '\x11'),
 (18, '\x12'),
 (19, '\x13'),
 (20, '\x14'),
 (21, '\x15'),
 (22, '\x16'),
 (23, '\x17'),
 (24, '\x18'),
 (25, '\x19'),
 (26, '\x1a'),
 (27, '\x1b'),
 (28, '\x1c'),
 (29, '\x1d'),
 (30, '\x1e'),
 (31, '\x1f'),
 (32, ' '),
 (33, '!'),
 (34, '"'),
 (35, '#'),
 (36, '$'),
 (37, '%'),
 (38, '&'),
 (39, "'"),
 (40, '('),
 (41, ')'),
 (42, '*'),
 (43, '+'),
 (44, ','),
 (45, '-'),
 (46, '.'),
 (47, '/'),
 (48, '0'),
 (49, '1'),
 (50, '2'),
 (51, '3'),
 (52, '4'),
 (53, '5'),
 (54, '6'),
 (55, '7'),
 (56, '8'),
 (57, '9'),
 (58, ':'),
 (59, ';'),
 (60, '<'),
 (61, '='),
 (62, '>'),
 (63, '?'),
 (64, '@'),
 (65, 'A'),
 (66, 'B'),
 (67, 'C'),
 (68, 'D'),
 (69, 'E'),
 (70, 'F'),
 (71, 'G'),
 (72, 'H'),
 (73, 'I'),
 (74, 'J'),
 (75, 'K'),
 (76, 'L

In [26]:
t2n['\x09']

9

In [10]:
len(take(10, t2c.items())[0][1])

388

In [28]:
import torch
from torch import sparse


In [29]:
codes = torch.from_numpy(tables[0])

In [30]:
# from https://discuss.pytorch.org/t/how-to-convert-a-dense-matrix-to-a-sparse-one/7809

def to_sparse(x):
    """ converts dense tensor x to sparse format """
    x_typename = torch.typename(x).split('.')[-1]
    sparse_tensortype = getattr(torch.sparse, x_typename)

    indices = torch.nonzero(x)
    if len(indices.shape) == 0:  # if all elements are zeros
        return sparse_tensortype(*x.shape)
    indices = indices.t()
    values = x[tuple(indices[i] for i in range(indices.shape[0]))]
    return sparse_tensortype(indices, values, x.size())


In [31]:
scodes = to_sparse(codes)

In [33]:
scodes.is_sparse

True

In [40]:
type(scodes)

torch.sparse.DoubleTensor

In [43]:
# pytorch sparse can't be saved yet ... not implemented for the moment (I should do it myself and send the patch)
# torch.save(scodes, "utf8-codes.pt")
# save_obj(scodes, "utf8-codes.torch")

In [11]:
import scipy as sp
import scipy.sparse

In [12]:
spcodes = sp.sparse.coo_matrix(tables[0])

In [13]:
save_obj(spcodes, "utf8-codes-scipy-sparse_3seg")

So, for the moment we have the posibility to encode all utf-8 characters, but is still a bit big in size when having the complete. But I'll try to cut the use of memory because 6.8GB for the "dense" matrix reprsentation is too much. In Sparse mode matrix is only 83MB for the entire dataset. Nevertheless there are many characters that I will not be using for the first tests, so having it use only a part will (should) be enough.

So I'll see how big the encoder is without the 4 segments, but only using the first 3 (this should be enough for most applications) so we can encode 

number of codes =  59328

number of code_exceptions =  4224

the entire code is now 206MB on a file on disk in non sparse mode and 3.6MB on sparse mode on disk. 

But also reducing the number of bytes on the code (using only 3 bytes max instead of 4) by not taking the last one that anyways we are not using for this application we can reduce this to 177MB of the complete "dense" code on disk and 3.6MB on sparse mode.

I would not recomend doing this all the time as it restricts the power of the input network to only known elements (and we want to do with all the possible codes) but for my tests this reduces the usage of memory, the amount of parameters and the processing time.

So I can start playing with it without worrying about memory ;)



In [10]:
type(c2t)

dict